<a href="https://colab.research.google.com/github/MuhammadSafeer786/Multimodal-AI-for-Myocardial-Infarction-or-Heart-Attack/blob/main/Gathering_ECG_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gathering Initial ECG Dataset


Using some images from this dataset: https://www.kaggle.com/datasets/jayaprakashpondy/ecgimages

# Preprocessing
Used the follwing script to Remove All Background

In [ ]:
import cv2
import numpy as np
import os
from skimage.morphology import skeletonize


def imshow(img, name):
    cv2.imshow(f'{name}', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


def ECGtoWave(image):
    blurred = cv2.GaussianBlur(image, (5, 5), 0)
    edges = cv2.Canny(blurred, threshold1=50, threshold2=150)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (8, 8))
    dilated = cv2.dilate(edges, kernel, iterations=1)
    eroded = cv2.erode(dilated, kernel, iterations=1)
    return eroded


def ECGtoWave2(image):
    _, binary = cv2.threshold(image, 35, 255, cv2.THRESH_BINARY)
    return binary


def Preprocess(input_dir, output_dir):
    img_list = os.listdir(input_dir)
    for img in img_list:
        image = cv2.imread(f'{input_dir}\{img}', cv2.IMREAD_GRAYSCALE)
        cleaned = ECGtoWave2(image)
        cv2.imwrite(f"{output_dir}\{img}", cleaned)





if __name__ == '__main__':
    Preprocess(r'\\ECG_Data\test\MI',               # Replace with your own path
               r'\\ECG\ECG_Data\test_cleaned\MI')   # Replace with your own path


# Annotated on Roboflow


ECG for MI > 2024-12-29 8:38pm

https://universe.roboflow.com/ecg-ziwon/ecg-for-mi

Provided by a Roboflow user

License: CC BY 4.0

# Exploring Dataset

In [ ]:
! mkdir /content/drive/MyDrive/FeaturedProjects/ECG_Dataset

In [1]:
!mv /content/ECG_MI_coco.zip /content/drive/MyDrive/FeaturedProjects/ECG_Dataset


In [4]:
!unzip /content/drive/MyDrive/FeaturedProjects/ECG_Dataset/ECG_MI_coco.zip -d /content/

Archive:  /content/drive/MyDrive/FeaturedProjects/ECG_Dataset/ECG_MI_coco.zip
  inflating: /content/README.dataset.txt  
  inflating: /content/README.roboflow.txt  
   creating: /content/test/
 extracting: /content/test/MI-42-_jpg.rf.f5bcf6146ae58dd64609a1df7ccb0080.jpg  
 extracting: /content/test/MI-54-_jpg.rf.778a84e9875955cb75289c4dfc197c3e.jpg  
 extracting: /content/test/MI-60-_jpg.rf.f6cbe3abaedbecfeec189c0bb33576b0.jpg  
 extracting: /content/test/Normal-36-_jpg.rf.f2930f921808dce3d78cb81b3506116f.jpg  
 extracting: /content/test/Normal-37-_jpg.rf.283332212718e10915a8a4bf7d2c467c.jpg  
 extracting: /content/test/Normal-57-_jpg.rf.221f37d60dfd1d3e8552410be436da67.jpg  
  inflating: /content/test/_annotations.coco.json  
   creating: /content/train/
 extracting: /content/train/MI-31-_jpg.rf.0861034f3893a842bda9cc0b5dc5794c.jpg  
 extracting: /content/train/MI-31-_jpg.rf.b246bac1b42254a70b62f405dcfe54e5.jpg  
 extracting: /content/train/MI-31-_jpg.rf.f3f854e91f4df7ba8cc887e92a370e

In [3]:
# Checking no. of files in these directories
!ls -1 '/content/train'| wc -l
!ls -1 '/content/test'| wc -l
!ls -1 '/content/valid'| wc -l

145
7
7


# Training Faster R-CNN

## Imports

In [13]:
import torch
import torchvision
from torchvision.transforms import Compose, Resize, ToTensor
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torch.utils.data import DataLoader
from pycocotools.coco import COCO
from PIL import Image
import os

## COCO Dataset Loader

In [25]:
class COCODataset(torch.utils.data.Dataset):
    def __init__(self, root, annotation_file, transform):
        self.root = root
        self.coco = COCO(annotation_file)
        self.ids = list(self.coco.imgs.keys())
        self.transform = transform

    def __getitem__(self, idx):
        # Load image
        image_info = self.images[idx]
        image_path = os.path.join(self.root_dir, image_info['file_name'])
        image = Image.open(image_path).convert("L")  # Grayscale

        # Get annotations for this image
        image_id = image_info['id']
        annotations = [ann for ann in self.annotations if ann['image_id'] == image_id]

        # Prepare bounding boxes and labels
        boxes = []
        labels = []
        for ann in annotations:
            x_min, y_min, width, height = ann['bbox']
            x_max = x_min + width
            y_max = y_min + height
            boxes.append([x_min, y_min, x_max, y_max])
            labels.append(ann['category_id'])  # Adjust as needed for your categories

        # Convert to tensors
        boxes = torch.tensor(boxes, dtype=torch.float32)
        labels = torch.tensor(labels, dtype=torch.int64)

        target = {
            "boxes": boxes,
            "labels": labels
        }

        # Apply transformations
        if self.transform:
            image = self.transform(image)

        return image, target

    def __len__(self):
        return len(self.ids)


## Load Dataset in COCO Format

In [26]:
transform = Compose([ToTensor()])

train_dataset = COCODataset('/content/train', '/content/train/_annotations.coco.json', transform=transform)
train_data_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))

test_dataset = COCODataset('/content/test', '/content/test/_annotations.coco.json', transform=transform)
test_data_loader = DataLoader(test_dataset, batch_size=4, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))

valid_dataset = COCODataset('/content/valid', '/content/valid/_annotations.coco.json', transform=transform)
valid_data_loader = DataLoader(valid_dataset, batch_size=4, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


## Load the Pre-trained Model

In [6]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes=2)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100%|██████████| 160M/160M [00:01<00:00, 90.7MB/s]


In [27]:
# Device Setup
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# Optimizer and Learning Rate Scheduler
optimizer = torch.optim.SGD(model.parameters(), lr=0.005, momentum=0.9)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

# Training Loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    for images, targets in train_data_loader:
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        # Forward pass
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        epoch_loss += losses.item()

        # Backward pass
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

    lr_scheduler.step()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")


AttributeError: 'COCODataset' object has no attribute 'images'